# Affichage des données téléphoniques

## Imports

In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [1]:


import folium
import pandas as pd;
from useful_methods.ihm.maps import mapUtils
from useful_methods.ihm.maps import map_ihm
import branca
from copy import deepcopy
import branca
import folium
from selenium import webdriver
from PIL import Image
import io
from selenium.webdriver.chrome.service import Service
from pathlib import Path
from PIL import Image
import os
from pyproj import Transformer
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn.cluster import HDBSCAN
import numpy as np


from geopy import distance

import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Importer le module depuis `useful_methods/criterias.py`
from useful_methods.neighbours_delaunay.simple_criterias import km_distance


## Plotting and saving subplots of maps

In [2]:
ntechs = len(map_ihm.technologies)
nprovs = len(map_ihm.providers)

Fonction pour créer une image à partir d'un fichier html 

In [3]:
def save_png(html_file, png_file, compressing_factor = 1):      
    # Configurer Selenium WebDriver
    service = Service(executable_path=r'/usr/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(2760, 1600)
    # Charger la page HTML dans le navigateur
    driver.get(f"file://{str(Path().absolute())}/{html_file}")

    # Prendre une capture d'écran de l'élément
    screenshot = driver.get_screenshot_as_png()
    driver.quit()

    # Convertir la capture d'écran en image avec Pillow
    image = Image.open(io.BytesIO(screenshot))

    # Compresser l'image
    width, height = image.size
    new_size = (width//compressing_factor, height//compressing_factor)
    image = image.resize(new_size)

    # Enregistrer l'image en PNG
    image.save(png_file)

Traçage des comparaisons des différents opérateurs pour chaque technologie

In [4]:
for technology in map_ihm.technologies :
    fig = branca.element.Figure()
    for i, provider in enumerate(map_ihm.providers) :
        subplot = fig.add_subplot(int(nprovs/2), nprovs-int(nprovs/2), i+1)
        subplot.add_child(mapUtils.getMap(deepcopy(map_ihm.df), [provider], map_ihm.regions, [technology], "Opérateurs", location = [46.8, 2.3], zoom_start = 6)) 
    mapUtils.addLegend(fig,{'Bouygues Telecom':'#035afc', 'Orange':'#fc5603', 'SFR':'#169e26', 'Free Mobile':'#dbd640'})
    fig.save(f"../../out/maps/providers-{technology}.html")
    save_png(f"../../out/maps/providers-{technology}.html", f"../../out/images/providers-{technology}.png", 2)

Traçage des répartitions des stations de base selon les opérateurs

In [5]:
fig = branca.element.Figure()
for i,provider in enumerate(map_ihm.providers) :
    subplot = fig.add_subplot(int(nprovs/2), nprovs-int(nprovs/2), i+1)
    subplot.add_child(mapUtils.getMap(deepcopy(map_ihm.df), [provider], map_ihm.regions, map_ihm.technologies, "Opérateurs", location = [46.8, 2.3], zoom_start = 6)) 
mapUtils.addLegend(fig,{'Bouygues Telecom':'#035afc', 'Orange':'#fc5603', 'SFR':'#169e26', 'Free Mobile':'#dbd640'})
fig.save("../../out/maps/subplots-operators.html")
save_png("../../out/maps/subplots-operators.html", "../../out/images/subplots-operators.png", 2)

### Couverture ciblée

In [6]:
data_couv_cible = pd.read_csv("../../database/2023_T4_dcc_Metropole_sites.csv", sep=";", decimal=',')
Nan_values_data_couv_cible = data_couv_cible[data_couv_cible['x_lambert_93'].isnull() | data_couv_cible['y_lambert_93'].isnull()]
data_couv_cible.dropna(axis=0, how='any', subset=['x_lambert_93', 'y_lambert_93'], inplace=True)


In [7]:
Nan_values_data_couv_cible[['x_lambert_93', 'y_lambert_93']]

,x_lambert_93,y_lambert_93
68,NaN,NaN
102,NaN,NaN
113,NaN,NaN
115,NaN,NaN
116,NaN,NaN
...,...,...
4548,NaN,NaN
4552,NaN,NaN
4553,NaN,NaN
4562,NaN,NaN


In [8]:
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")

# Définir le transformateur pour la conversion de Lambert 93 à WGS84
transformer = Transformer.from_crs("epsg:2154", "epsg:4326")

for ind, x, y in data_couv_cible[['x_lambert_93', 'y_lambert_93']].itertuples():
    # Convertir les coordonnées Lambert 93 en longitude/latitude
    latitude, longitude = transformer.transform(x, y)
    map.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], fill_opacity=0.2, radius=1.5, popup=f"({x}, {y})"))
map.save("../../out/maps/couverture_ciblee.html")
save_png("../../out/maps/couverture_ciblee.html","../../out/images/couverture_ciblee.png", 2)

## H-DBScan

In [9]:
def probaCity(coordsXY):
    clusterer= HDBSCAN(min_cluster_size=5, min_samples=40)
    clusterer.fit(coordsXY)
    return pd.Series(data = clusterer.probabilities_, index=coordsXY.index)

In [10]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')
df_filtered = df[df['nom_op']=='Orange']
positions = df_filtered[['x','y']]
positionsLl = df_filtered[['longitude','latitude']]

In [11]:
probabilities = probaCity(positions)

In [12]:
# Create a colormap
cmap = plt.cm.get_cmap('winter_r')  # 'cool' colormap for blue to gray
norm = mcolors.Normalize(vmin=0, vmax=1)

# Create a map centered on France
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")

for ind, longitude, latitude in positionsLl.itertuples():
    proba = probabilities[ind]
    color = 'gray' if proba == 0 else mcolors.to_hex(cmap(norm(proba)))
    map.add_child(folium.RegularPolygonMarker(location=[latitude, longitude], color=color, radius=1.5, popup=f"proba={probabilities[ind]}"))
map.save("../../out/maps/villes_HDBSCAN.html")
save_png("../../out/maps/villes_HDBSCAN.html","../../out/images/villes_HDBSCAN.png", 2)

## Afficher le graphe de voisinage

In [13]:
# Create a map centered on France
map = folium.Map(location=[46.8, 2.3], zoom_start=7, tiles="Cartodb Positron")